## U.S. Bureau of Labor Statistics - CPI Analysis
#### Eric Bottinelli

### 1. Retrieve data via BLS API v2

**Documentation**

- https://www.bls.gov/developers/api_python.htm
- https://data.bls.gov/cgi-bin/surveymost?cu
- https://data.bls.gov/dataQuery/find?fq=survey:[cu]&s=popularity:D&r=100&st=0
- https://www.bls.gov/cpi/tables/relative-importance/2023.htm

**Packages to install**

- Prettytable ('pip install prettytable')

**API Series ID**

Consumer Price Index for All Urban Consumers (CPI-U)
- *All items in U.S. city average, all urban consumers*
    - NSA: CUUR0000SA0
    - SA: CUSR0000SA0
- *All items less food and energy in U.S. city average, all urban consumers*
    - NSA: CUUR0000SA0L1E
    - SA: CUSR0000SA0L1E
- *Food in U.S. city average, all urban consumers*
    - NSA: CUUR0000SAF1
    - SA: CUSR0000SAF1
- *Food at home in U.S. city average, all urban consumers*
    - NSA: CUUR0000SAF11
    - SA: CUSR0000SAF11
- *Energy in U.S. city average, all urban consumers*
    - NSA: CUUR0000SA0E
    - SA: CUSR0000SA0E
- *Commodities less food and energy commodities in U.S. city average, all urban consumers*
    - NSA: CUUR0000SACL1E
    - SA: CUSR0000SACL1E4
- *Services less energy services in U.S. city average, all urban consumers*
    - NSA: CUUR0000SASLE
    - SA: CUSR0000SASLE
- *Shelter in U.S. city average, all urban consumers*
    - NSA: CUUR0000SAH1
    - SA: CUSR0000SAH1
((https://www.bls.gov/cpi/factsheets/owners-equivalent-rent-and-rent.htm))

**Calculate special CPI**

Occasionally, a user wishes to estimate a price change that is not published by BLS. For instance, suppose a user would like a CPI series for ‘services less energy services and shelter’. This can be done by estimating a special index, in this case, ‘services less energy services and shelter’.
[BLS Doc](https://www.bls.gov/cpi/factsheets/constructing-special-cpis.htm)

If SEEB01 -> CUUR0000SEEB01

Cost weight is just a sum of all the items

If I add all the values to calculate the services less energy services and shelter, it becomes a lot of data. Explore different solution (e.g. remove goods from core CPI)

**Supercore CPI**

"Fed Chair Jerome Powell cited a specific category of inflation—inflation in core services other than housing—as being perhaps “the most important category for understanding the future evolution of core inflation.” The financial press has termed this category “supercore” inflation" ([FED of St. Louis](https://www.stlouisfed.org/on-the-economy/2024/may/measuring-inflation-headline-core-supercore-services))

In [2]:
import os
import requests
import json
import prettytable
import pandas as pd
from datetime import datetime

folder_name = 'CPI_Data'

food_weight = 13.555
energy_weight = 6.655
services_less_energy_weight = 60.899
shelter_weight = 36.191

weight_map = {
    'All_Items': 100,
    'Food_Energy': food_weight + energy_weight,
    'Food': food_weight,
    'Food_At_Home': 8.167,
    'Food_Away_From_Home': 5.388,
    'Energy': energy_weight,
    'All_Items_Less_Food_Energy': 79.790,
    'Commodities_Less_Food_Energy_Commodities': 18.891,
    'Services_Less_Energy_Services': services_less_energy_weight,
    'Shelter': shelter_weight,
    'Supercore': services_less_energy_weight - shelter_weight 
}

In [165]:
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

current_date = datetime.now()
current_year = current_date.year
last_year = current_year - 1

series_names = {
    'CUUR0000SA0': 'NSA_All_Items',
    'CUSR0000SA0': 'SA_All_Items',
    'CUUR0000SA0L1E': 'NSA_All_Items_Less_Food_Energy',
    'CUSR0000SA0L1E': 'SA_All_Items_Less_Food_Energy',
    'CUUR0000SAF': 'NSA_Food',
    'CUSR0000SAF': 'SA_Food',
    'CUUR0000SAF11': 'NSA_Food_At_Home',
    'CUSR0000SAF11': 'SA_Food_At_Home',
    'CUUR0000SEFV': 'NSA_Food_Away_From_Home',
    'CUSR0000SEFV': 'SA_Food_Away_From_Home',
    'CUUR0000SA0E': 'NSA_Energy',
    'CUSR0000SA0E': 'SA_Energy',
    'CUUR0000SACL1E': 'NSA_Commodities_Less_Food_Energy_Commodities',
    'CUSR0000SACL1E4': 'SA_Commodities_Less_Food_Energy_Commodities',
    'CUUR0000SASLE': 'NSA_Services_Less_Energy_Services',
    'CUSR0000SASLE': 'SA_Services_Less_Energy_Services',
    'CUUR0000SAH1': 'NSA_Shelter',
    'CUSR0000SAH1': 'SA_Shelter',
}
series_ids = list(series_names.keys())

headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": series_ids, "startyear": str(last_year), "endyear": str(current_year)})
response = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(response.text)

all_data = []
for series in json_data['Results']['series']:
    rows = []
    for item in series['data']:
        footnotes = "".join([footnote['text'] + ',' for footnote in item['footnotes'] if footnote]).rstrip(',')
        if 'M01' <= item['period'] <= 'M12':
            rows.append([series_names[series['seriesID']], item['year'], item['period'], item['value'], footnotes])

    df = pd.DataFrame(rows, columns=["series id", "year", "period", "value", "footnotes"])
    all_data.append(df)

complete_data = pd.concat(all_data)

df = complete_data.copy()
df['date'] = pd.to_datetime(df['year'].astype(str) + df['period'].str.replace('M', ''), format='%Y%m')
df['series id'] = df['series id'].astype(str) 
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df['footnotes'] = df['footnotes'].astype(str) 
df.drop(['year', 'period', 'footnotes'], axis=1, inplace=True)
df.rename(columns={'series id': 'id'}, inplace=True)
df = df[['id', 'date', 'value']]

csv_path = os.path.join(folder_name, 'CPI_data.csv')
df.to_csv(csv_path, index=False)

In [3]:
df = pd.read_csv("CPI_Data/CPI_data.csv")
df['date'] = pd.to_datetime(df['date'])

In [4]:
df['MoM_change'] = df.groupby('id')['value'].transform(lambda x: (x - x.shift(-1)) / x.shift(-1))
df['YoY_change'] = df.groupby('id')['value'].transform(lambda x: (x - x.shift(-12)) / x.shift(-12))

In [5]:
def calculate_weighted_change(df, id1, id2, new_id, weight1, weight2, weight_total, operation): 
    series1 = df.loc[df['id'] == id1, ['MoM_change', 'YoY_change']].set_index(df.loc[df['id'] == id1, 'date']) * weight1
    series2 = df.loc[df['id'] == id2, ['MoM_change', 'YoY_change']].set_index(df.loc[df['id'] == id2, 'date']) * weight2

    if operation == 'add':
        result = (series1 + series2) / weight_total
    elif operation == 'subtract':
        result = (series1 - series2) / weight_total
    else:
        raise ValueError("Operation must be 'add' or 'subtract'.")

    result = result.reset_index()
    result['id'] = new_id
    result['value'] = 0

    result = result[['id', 'date', 'value', 'MoM_change', 'YoY_change']]
    return result

# Calculate 'SA_Food_Energy' and 'NSA_Food_Energy'
df = pd.concat([
    df,
    calculate_weighted_change(df, 'SA_Food', 'SA_Energy', 'SA_Food_Energy', weight_map['Food'], weight_map['Energy'], weight_map['Food_Energy'], 'add'),
    calculate_weighted_change(df, 'NSA_Food', 'NSA_Energy', 'NSA_Food_Energy', weight_map['Food'], weight_map['Energy'], weight_map['Food_Energy'], 'add')
])

# Calculate 'SA_Supercore' and 'NSA_Supercore'
df = pd.concat([
    df,
    calculate_weighted_change(df, 'SA_Services_Less_Energy_Services', 'SA_Shelter', 'SA_Supercore', weight_map['Services_Less_Energy_Services'], weight_map['Shelter'], weight_map['Supercore'], 'subtract'),
    calculate_weighted_change(df, 'NSA_Services_Less_Energy_Services', 'NSA_Shelter', 'NSA_Supercore', weight_map['Services_Less_Energy_Services'], weight_map['Shelter'], weight_map['Supercore'], 'subtract')
])

In [12]:
category_map = {
    'All_Items': (0, 'Headline', '', '', 'All Items'),
    'Food_Energy': (1, 'Food + Energy', '', '', 'Food and Energy'),
    'Food': (2, '', 'Food', '', 'Food'),
    'Food_At_Home': (3, '', '', 'At home', 'Food at Home'),
    'Food_Away_From_Home': (4, '', '', 'Away Home', 'Food Away from Home'),
    'Energy': (5, '', 'Energy', '', 'Energy'),
    'All_Items_Less_Food_Energy': (6, 'Core', '', '', 'Core CPI excluding Food and Energy'),
    'Commodities_Less_Food_Energy_Commodities': (7, '', 'Commodities', '', 'Commodities excluding Food and Energy'),
    'Services_Less_Energy_Services': (8, '', 'Services', '', 'Services excluding Energy'),
    'Shelter': (9, '', '', 'Shelter', 'Shelter'),
    'Supercore': (10, '', '', 'Supercore', 'Supercore')
}

ordered_categories = ['Headline', 'Food + Energy', 'Core']
ordered_sub_categories_1 = ['Food', 'Energy', 'Commodities', 'Services']
ordered_sub_categories_2 = ['At home', 'Away Home', 'Shelter', 'Supercore']

for i in range(4):
    if i in [0, 1]:
        data = df[df['id'].str.startswith('NSA_')].copy()
        prefix_length = 4
    else:
        data = df[df['id'].str.startswith('SA_')].copy() 
        prefix_length = 3

    data.loc[:, 'id'] = data['id'].str[prefix_length:]

    missing_ids = set(data['id']) - set(category_map.keys())
    if missing_ids:
        raise ValueError(f"Missing IDs in category_map: {missing_ids}")

    data.loc[:, 'Month-Year'] = data['date'].dt.strftime('%b-%y')
    order_cat = data['id'].apply(lambda x: category_map.get(x, (None, None, None, None, None)))
    data.loc[:, 'Order'] = [item[0] for item in order_cat]
    data.loc[:, 'Category'] = [item[1] for item in order_cat]
    data.loc[:, 'Sub Category 1'] = [item[2] for item in order_cat]
    data.loc[:, 'Sub Category 2'] = [item[3] for item in order_cat]
    data.loc[:, 'Name'] = [item[4] for item in order_cat]
    data.loc[:, 'Weight'] = data['id'].map(weight_map).fillna('Unknown')

    if i in [0, 2]:
        value_column = 'MoM_change'
    else:
        value_column = 'YoY_change'

    data_pivot = data.pivot_table(
        index=['Name', 'Order', 'Category', 'Sub Category 1', 'Sub Category 2', 'Weight'],
        columns='Month-Year',
        values=value_column,
        aggfunc='first'
    )

    data_pivot = data_pivot[sorted(data_pivot.columns, key=lambda x: pd.to_datetime(x, format='%b-%y'), reverse=True)]
    data_pivot.columns.name = None
    data_pivot.reset_index(inplace=True)
    data_pivot.sort_values(by='Order', inplace=True)
    data_pivot.drop(columns=['Order'], inplace=True)

    if i == 0:
        NSA_MoM_df = data_pivot
    elif i == 1:
        NSA_YoY_df = data_pivot
    elif i == 2:
        SA_MoM_df = data_pivot
    else:
        SA_YoY_df = data_pivot

In [13]:
NSA_MoM_df.head()

,Name,Category,Sub Category 1,Sub Category 2,Weight,Jul-24,Jun-24,May-24,Apr-24,Mar-24,...,Nov-23,Oct-23,Sep-23,Aug-23,Jul-23,Jun-23,May-23,Apr-23,Mar-23,Feb-23
0,All Items,Headline,,,100.000,0.001162,0.000338,0.001662,0.003893,0.006464,...,-0.002015,-0.000383,0.002485,0.004367,0.001908,0.003229,0.002518,0.005059,0.003311,0.005582
6,Food and Energy,Food + Energy,,,20.210,0.003035,-0.002701,0.000198,0.007922,0.010954,...,-0.012327,-0.008182,0.003275,0.012149,0.003016,0.005400,-0.002583,0.006610,-0.002419,0.000923
4,Food,,Food,,13.555,0.002482,0.001834,0.001343,0.001893,0.000926,...,-0.001729,0.003144,0.002087,0.001738,0.002811,0.000966,0.002275,0.002347,0.000906,0.004247
7,Food at Home,,,At home,8.167,0.002914,0.000239,-0.000092,0.000920,-0.000141,...,-0.005131,0.002840,0.000688,0.000860,0.003704,-0.000661,0.000685,0.001358,-0.001868,0.003477
5,Food Away from Home,,,Away Home,5.388,0.002081,0.004132,0.003515,0.003476,0.002613,...,0.004345,0.003737,0.003946,0.003441,0.001742,0.003834,0.004712,0.003695,0.005965,0.006341


In [14]:
csv_path = os.path.join(folder_name, 'NSA_MoM_CPI_data.csv')
NSA_MoM_df.to_csv(csv_path, index=False)

csv_path = os.path.join(folder_name, 'NSA_YoY_CPI_data.csv')
NSA_YoY_df.to_csv(csv_path, index=False)

csv_path = os.path.join(folder_name, 'SA_MoM_CPI_data.csv')
SA_MoM_df.to_csv(csv_path, index=False)

csv_path = os.path.join(folder_name, 'SA_YoY_CPI_data.csv')
SA_YoY_df.to_csv(csv_path, index=False)